In [31]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from sklearn.exceptions import UndefinedMetricWarning
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor

# Desactivar los warnings UndefinedMetricWarning para r2_score
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [32]:
def preparar_datos(df, tb_productos, lags=12):
    # Convertir el periodo a formato datetime
    df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

    # Agregar los datos por periodo y product_id para obtener la serie temporal
    ts = df.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

    # Unir las categorías de productos desde el archivo tb_productos
    ts = ts.merge(tb_productos[['product_id', 'cat1', 'cat2', 'cat3']], on='product_id', how='left')

    # Convertir las columnas de categoría a tipo 'category'
    ts['cat1'] = ts['cat1'].astype('category')
    ts['cat2'] = ts['cat2'].astype('category')
    ts['cat3'] = ts['cat3'].astype('category')
    
    # Crear características adicionales
    ts['crisis'] = (ts['periodo'].dt.year == 2019) & (ts['periodo'].dt.month == 8)
    ts['quarter'] = ts['periodo'].dt.quarter
    ts['month'] = ts['periodo'].dt.month

    # Normalización por producto
    ts['tn_norm'] = ts.groupby('product_id')['tn'].transform(lambda x: (x - x.mean()) / x.std())

    # Agregar lags a los datos
    for lag in range(1, lags + 1):
        ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

    return ts

In [33]:
def entrenar_modelo(ts, lags=12):
    # Crear conjunto de entrenamiento y objetivo
    X = ts[['product_id', 'cat1', 'cat2','cat3','crisis', 'quarter', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)] + ['tn_norm']]
    y = ts['tn'].shift(-2)

    # Eliminar las últimas 2 filas
    X = X.iloc[:-2]
    y = y.iloc[:-2]

    # Validación temporal en lugar de train_test_split
    tscv = TimeSeriesSplit(n_splits=5)
    X_train, X_test, y_train, y_test = None, None, None, None
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Codificar las características categóricas 'cat1', 'cat2', 'cat3'
    for col in ['cat1', 'cat2', 'cat3']:
        X_train[col] = X_train[col].cat.codes
        X_test[col] = X_test[col].cat.codes

    # Definir el espacio de búsqueda de hiperparámetros
    param_dist = {
        'num_leaves': [31, 50, 70, 128],
        'max_depth': [-1, 10, 20, 30],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 500],
        'min_child_samples': [20, 30, 40],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    # Definir el modelo de LightGBM con RandomizedSearchCV
    lgb_model = lgb.LGBMRegressor(random_state=42)
    random_search = RandomizedSearchCV(lgb_model, param_distributions=param_dist, n_iter=100, cv=5, verbose=1, n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)

    print(f"Best parameters found: {random_search.best_params_}")

    # Crear y ajustar el modelo de Random Forest
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # Crear y ajustar el modelo de XGBoost
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_model.fit(X_train, y_train)

    # Obtener el mejor modelo de LightGBM
    lgb_model = random_search.best_estimator_

    # Crear el modelo de ensemble
    ensemble_model = VotingRegressor(estimators=[
        ('lgb', lgb_model),
        ('rf', rf_model),
        ('xgb', xgb_model)
    ])

    # Ajustar el modelo de ensemble
    ensemble_model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = ensemble_model.predict(X_test)

    # Calcular métricas de rendimiento
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Ensemble Model MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

    return ensemble_model


In [34]:
def predecir_producto(ensemble_model, ts, product_ids, next_period='2020-02-01', lags=12):
    next_period = pd.Timestamp(next_period)
    results = []

    for product_id in tqdm(product_ids, desc="Predicting with ensemble model"):
        product_data = ts[ts['product_id'] == product_id].copy()
        if not product_data.empty:
            last_data = product_data.iloc[-1]

            # Convertir a categoría si es list-like
            try:
                cat1 = pd.Categorical(last_data['cat1'])
                cat2 = pd.Categorical(last_data['cat2'])
                cat3 = pd.Categorical(last_data['cat3'])
            except TypeError:
                cat1, cat2, cat3 = None, None, None

            if cat1 is not None and cat2 is not None and cat3 is not None:
                # Construir datos para la predicción
                next_data = pd.DataFrame({
                    'product_id': [product_id],
                    'cat1': [cat1.codes[0] if len(cat1) > 0 else 0],
                    'cat2': [cat2.codes[0] if len(cat2) > 0 else 0],
                    'cat3': [cat3.codes[0] if len(cat3) > 0 else 0],
                    'crisis': [(next_period.year == 2019) & (next_period.month == 8)],
                    'quarter': [next_period.quarter],
                    'month': [next_period.month],
                    **{f'tn_lag_{lag}': [last_data[f'tn_lag_{lag}']] if f'tn_lag_{lag}' in product_data.columns else [0] for lag in range(1, lags + 1)},
                    'tn_norm': [0]  # Ajustar tn_norm adecuadamente si es necesario
                })

                # Predecir usando el modelo de ensemble
                pred = ensemble_model.predict(next_data)
                results.append({'product_id': product_id, 'predicted_tn': pred[0]})
            else:
                product_mean_tn = ts[ts['product_id'] == product_id]['tn'].mean()
                if not pd.isna(product_mean_tn):
                    results.append({'product_id': product_id, 'predicted_tn': product_mean_tn})
                else:
                    global_mean_tn = ts['tn'].mean()
                    results.append({'product_id': product_id, 'predicted_tn': global_mean_tn})
        else:
            product_mean_tn = ts[ts['product_id'] == product_id]['tn'].mean()
            if not pd.isna(product_mean_tn):
                results.append({'product_id': product_id, 'predicted_tn': product_mean_tn})
            else:
                global_mean_tn = ts['tn'].mean()
                results.append({'product_id': product_id, 'predicted_tn': global_mean_tn})

    return pd.DataFrame(results)

In [35]:
def evaluar_metricas(df, results_df, target_date='2019-12-01'):
    df['date'] = pd.to_datetime(df['periodo'], format='%Y%m')
    df_filtered = df[df['date'] == target_date]
    results_df_ajustado = results_df.groupby('product_id')['predicted_tn'].sum()

    metricas_por_producto = []
    numerador = 0
    denominator = 0

    for product_id in df_filtered['product_id'].unique():
        if product_id in results_df_ajustado.index:
            y_true = df_filtered.loc[df_filtered['product_id'] == product_id, 'tn'].values[0]
            y_pred = results_df_ajustado.loc[product_id]
            numerador += abs(y_true - y_pred)
            denominator += y_true
            rmse = np.sqrt(mean_squared_error([y_true], [y_pred]))
            mae = mean_absolute_error([y_true], [y_pred])
            r2 = r2_score([y_true], [y_pred]) if len([y_true]) > 1 and len([y_pred]) > 1 else float('nan')
            metricas_por_producto.append({
                'product_id': product_id,
                'rmse': rmse,
                'mae': mae,
                'r2': r2
            })

    if metricas_por_producto:
        avg_rmse = np.mean([m['rmse'] for m in metricas_por_producto])
        avg_mae = np.mean([m['mae'] for m in metricas_por_producto])
        avg_r2 = np.nanmean([m['r2'] for m in metricas_por_producto])
        metricas_por_producto.append({
            'product_id': 'average',
            'rmse': avg_rmse,
            'mae': avg_mae,
            'r2': avg_r2
        })

    metricas_df = pd.DataFrame(metricas_por_producto)
    metricaMultinacion = numerador / denominator
    print("Métrica multinacional", metricaMultinacion)
    print("metrica multinacional", metricaMultinacion)
    print("rmse: ", metricas_df['rmse'].mean())
    print("mae: ", metricas_df['mae'].mean())
    print("r2: ", metricas_df['r2'].mean())

    # Exportar métricas a un archivo CSV
    metricas_df.to_csv('metricas_por_producto.csv', index=False)
    print(f"Métricas por producto exportadas a 'metricas_por_producto.csv'")
    
    return metricas_df

Inicio codigo

In [36]:
# Cargar los datos
df = pd.read_csv('../../../sell-in.txt/sell-in.txt', sep='\t')
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')
tb_productos = pd.read_csv('c:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/tb_productos_descripcion.txt', sep='\t')

In [37]:
ts = preparar_datos(df, tb_productos, lags=12)

Predicción sobre febrero 2020

In [38]:
productosPredecir = ts['product_id'].values
productosPredecir

array([20001, 20002, 20003, ..., 21267, 21271, 21276], dtype=int64)

In [39]:
# Filtro por producto 20005 y periodo menor a 2019-11-01
#productoPrueba = [20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,20011,20012]
# Convertir cada elemento a int64
#productoPrueba_int64 = [np.int64(x) for x in productosPredecir]
#ts = ts[ts['product_id'] == productoPrueba].copy()
#ts = ts[ts['periodo'] < '2019-11-01']

In [40]:
ensemble_model = entrenar_modelo(ts, lags=12)

C:\Users\Josvaldes\AppData\Local\Temp\ipykernel_31512\2656096673.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].cat.codes
C:\Users\Josvaldes\AppData\Local\Temp\ipykernel_31512\2656096673.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].cat.codes


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005838 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3695
[LightGBM] [Info] Number of data points in the train set: 26035, number of used features: 19
[LightGBM] [Info] Start training from score 44.668985
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [41]:
result_df = predecir_producto(ensemble_model, ts, productosPredecir, next_period='2020-02-01', lags=12)

Predicting with ensemble model: 100%|██████████| 31243/31243 [00:54<00:00, 576.45it/s]


In [42]:
print(result_df)

       product_id  predicted_tn
0           20001   1398.344322
1           20002   1009.368178
2           20003    889.004243
3           20004    671.615383
4           20005    644.200514
...           ...           ...
31238       21265      0.089541
31239       21266      0.094659
31240       21267      0.092835
31241       21271      0.026964
31242       21276      0.045447

[31243 rows x 2 columns]


In [43]:
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')

# Asegúrate de que la columna 'product_id' en ambos DataFrames sea del mismo tipo
result_df['product_id'] = result_df['product_id'].astype(int)
productosPredecir['product_id'] = productosPredecir['product_id'].astype(int)

# Realiza un merge para obtener solo los productos predichos que están en productosPredecir
predicted_products = pd.merge(productosPredecir, result_df, on='product_id', how='inner')

# Eliminar duplicados para asegurarse de tener un producto único
predicted_products = predicted_products.drop_duplicates(subset=['product_id'])

# Verifica el resultado
print(predicted_products)

       product_id  predicted_tn
0           20001   1398.344322
36          20002   1009.368178
72          20003    889.004243
108         20004    671.615383
144         20005    644.200514
...           ...           ...
22294       21263      0.089233
22309       21265      0.089541
22319       21266      0.094659
22329       21267      0.092835
22339       21276      0.045447

[780 rows x 2 columns]


In [44]:
predicted_products.to_csv('resultadosPredichos.csv', index=False)

Validación sobre diciembre 2019

In [54]:
ts = preparar_datos(df, tb_productos, lags=12)
ts = ts[ts['periodo'] < '2019-11-01']

In [46]:
# Convertir cada elemento a int64
#productoPrueba_int64 = [np.int64(x) for x in productosPredecir]

In [55]:
ensemble_model = entrenar_modelo(ts, lags=12)

C:\Users\Josvaldes\AppData\Local\Temp\ipykernel_31512\2656096673.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].cat.codes
C:\Users\Josvaldes\AppData\Local\Temp\ipykernel_31512\2656096673.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].cat.codes


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000941 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3694
[LightGBM] [Info] Number of data points in the train set: 24480, number of used features: 19
[LightGBM] [Info] Start training from score 44.852583
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Best parameters found: {'subsample': 0.8, 'num_leaves': 128, 'n_estimators': 100, 'min_child_samples': 20, 'max_depth': 20, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001330 seconds.
You can set `force_col_wise=t

In [59]:
result_df = predecir_producto(ensemble_model, ts, productosPredecir, next_period='2019-12-01', lags=12)

Predicting with ensemble model:   0%|          | 1/780 [00:00<00:03, 221.44it/s]


In [60]:
result_df

,product_id,predicted_tn
0,product_id,43.052861


In [50]:
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')

In [51]:
productosPredecir

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005
...,...
775,21263
776,21265
777,21266
778,21267


In [52]:
# Asegúrate de que la columna 'product_id' en ambos DataFrames sea del mismo tipo
result_df['product_id'] = result_df['product_id'].astype(int)
productosPredecir['product_id'] = productosPredecir['product_id'].astype(int)

# Realiza un merge para obtener solo los productos predichos que están en productosPredecir
predicted_products = pd.merge(productosPredecir, result_df, on='product_id', how='inner')

# Eliminar duplicados para asegurarse de tener un producto único
predicted_products = predicted_products.drop_duplicates(subset=['product_id'])

# Verifica el resultado
print(predicted_products)

ValueError: invalid literal for int() with base 10: 'product_id'

In [ ]:
ts = preparar_datos(df, tb_productos, lags=12)

In [ ]:
# Paso 1: Filtrar el DataFrame 'ts' para obtener los datos del período específico y los productos a predecir
filtered_df = ts[(ts['periodo'] == '2019-12-01') & (ts['product_id'].isin(productosPredecir['product_id']))]

# Paso 2: Agrupar los datos filtrados por 'product_id' y calcular la suma de 'tn' para cada producto
real_tn = filtered_df.groupby('product_id')['tn'].sum()

# Paso 3: Eliminar duplicados en 'result_df' para asegurar que cada producto aparezca una vez
result_df_unique = result_df.drop_duplicates(subset='product_id')

# Paso 4: Realizar un merge para asegurar que los 'product_id' coincidan en 'result_df' y 'real_tn'
result_df_unique = result_df_unique.merge(real_tn.rename('real_tn'), on='product_id', how='left')

# Paso 5: Calcular la métrica de la empresa por producto
result_df_unique['metricaempresa'] = abs(result_df_unique['real_tn'] - result_df_unique['predicted_tn']) / result_df_unique['real_tn']

# Paso 6: Filtrar 'result_df_unique' para obtener solo los productos que están en 'productosPredecir'
final_result_df = result_df_unique[result_df_unique['product_id'].isin(productosPredecir['product_id'])]

# Imprimir el resultado final
print(final_result_df)

      product_id  predicted_tn     real_tn  metricaempresa
0          20001   1395.245139  1504.68856        0.072735
1          20002    994.893190  1087.30855        0.084995
2          20003    887.157574   892.50129        0.005987
3          20004    671.067991   637.90002        0.051996
4          20005    646.795952   593.24443        0.090269
...          ...           ...         ...             ...
1185       20962      5.317000     1.99182        1.669418
1186       20975      4.981210     1.69045        1.946677
1187       20995      4.624140     1.55285        1.977841
1188       21087      0.603995     1.02205        0.409036
1189       21214      0.279925     0.24428        0.145919

[780 rows x 4 columns]


In [ ]:
metrica = evaluar_metricas(df, result_df, target_date='2019-12-01')

Métrica multinacional 5095.06489170038
metrica multinacional 5095.06489170038
rmse:  1306.1396166842176
mae:  1306.1396166842176
r2:  nan
Métricas por producto exportadas a 'metricas_por_producto.csv'


C:\Users\Josvaldes\AppData\Local\Temp\ipykernel_31512\4073115482.py:29: RuntimeWarning: Mean of empty slice
  avg_r2 = np.nanmean([m['r2'] for m in metricas_por_producto])


In [ ]:
final_result_df.to_csv('validacionDic23.csv', index=False)

Segunda iteración con productos que tiene una metrica de la empresa superior al 10%

In [ ]:
# Filtrar el DataFrame usando la función query
SegundaIteracion_result_df = final_result_df.query('metricaempresa > 0.10')

# Imprimir el resultado filtrado
print(SegundaIteracion_result_df)

      product_id  predicted_tn    real_tn  metricaempresa
5          20006    596.232753  417.23228        0.429019
6          20007    625.592909  390.43432        0.602300
7          20008    569.306666  195.36854        1.914014
9          20010    524.720959  359.59998        0.459180
10         20011    452.907964  392.38290        0.154250
...          ...           ...        ...             ...
1185       20962      5.317000    1.99182        1.669418
1186       20975      4.981210    1.69045        1.946677
1187       20995      4.624140    1.55285        1.977841
1188       21087      0.603995    1.02205        0.409036
1189       21214      0.279925    0.24428        0.145919

[689 rows x 4 columns]


In [ ]:
result_df

,product_id,predicted_tn
0,20001,1395.245139
1,20002,994.893190
2,20003,887.157574
3,20004,671.067991
4,20005,646.795952
...,...,...
31238,21265,0.097417
31239,21266,0.103531
31240,21267,0.109018
31241,21271,0.028482


In [ ]:
# Filtrar el DataFrame usando la función query
listadoProductosFinal_result_df_1 = final_result_df.query('metricaempresa < 0.10')

predicted_products = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Predicciones/resultadosPredichos.csv', sep='\t')

# Asegúrate de que la columna 'product_id' en ambos DataFrames sea del mismo tipo
predicted_products['product_id'] = predicted_products['product_id'].astype(int)
listadoProductosFinal_result_df_1['product_id'] = listadoProductosFinal_result_df_1['product_id'].astype(int)

# Realiza un merge para obtener solo los productos predichos que están en productosPredecir
listadoProductosFinal_result_df_1 = pd.merge(listadoProductosFinal_result_df_1, predicted_products, on='product_id', how='inner')

# Eliminar duplicados para asegurarse de tener un producto único
#listadoProductosFinal_result_df_1 = predicted_products.drop_duplicates(subset=['product_id'])

# Verifica el resultado
print(listadoProductosFinal_result_df_1)

      product_id  predicted_tn     real_tn  metricaempresa
0          20001   1395.245139  1504.68856        0.072735
1          20002    994.893190  1087.30855        0.084995
2          20003    887.157574   892.50129        0.005987
3          20004    671.067991   637.90002        0.051996
4          20005    646.795952   593.24443        0.090269
...          ...           ...         ...             ...
1160       21035      1.756772     1.80884        0.028785
1161       21039      2.100980     1.94374        0.080896
1167       20694      8.357723     8.90416        0.061369
1168       20762      6.762283     6.82849        0.009696
1174       21097      1.281353     1.34469        0.047101

[91 rows x 4 columns]
